# Data Visualisation Portfolio - 199110629

## Package Imports

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import plotly.express as px
import plotly.io as pio

# Data Imports

## Mapping shapefiles

In [2]:
# In theory, you could use the Open data portal API but it times out on this data for some reason so downloaded directly
with open('Local_Authority_Districts_(December_2021)_GB_BGC.geojson') as file:
    uk_districts = json.load(file)

## ONS house price to earnings

Data is sourced from the ONS dataset (House price to workplace-based earnings ratio) accessed from the [ONS website](https://www.ons.gov.uk/peoplepopulationandcommunity/housing/datasets/ratioofhousepricetoworkplacebasedearningslowerquartileandmedian) downloaded 09/05/2022. The datasource is provided in the repo but the history of publications is available from ONS.

In [ ]:
data = 'ratioofhousepricetoworkplacebasedearnings.xlsx'
#Region
price_by_region = pd.read_excel(data, '1a', skiprows=1, usecols='B:AA')
earnings_by_region = pd.read_excel(data, '1b', skiprows=1, usecols='B:AA')
ratio_by_region = pd.read_excel(data, '1c', skiprows=1, usecols='B:AA')
#County
price_by_district = pd.read_excel(data, '5a', skiprows=1, usecols='D:AC')
earnings_by_district = pd.read_excel(data, '5b', skiprows=1, usecols='D:AC').replace(':', np.NaN)
ratio_by_district = pd.read_excel(data, '5c', skiprows=1, usecols='D:AC').replace(':', np.NaN)

earnings_by_district

# Transformations

## Wide data

In [ ]:
def wide_ons_data(data_in,time_step,split_by,metric):
    data_out = data_in
    data_out.columns = data_out.columns.map(lambda x: str(x)[-4:])
    data_out = data_out.transpose()
    data_out.columns = data_out.iloc[0]
    data_out = data_out[1:].astype(float)
    data_out.index.names = [time_step]
    data_out.columns.names = [split_by]
    data_out = data_out.add_suffix(' '+metric)
    return data_out

#Region
pbr_wide = wide_ons_data(price_by_region, 'Year', 'Region', 'Price')
ebr_wide = wide_ons_data(earnings_by_region, 'Year', 'Region', 'Earnings')
rbr_wide = wide_ons_data(ratio_by_region, 'Year', 'Region', 'Ratio')
#County
pbd_wide = wide_ons_data(price_by_district, 'Year', 'District', 'Price')
ebd_wide = wide_ons_data(earnings_by_district, 'Year', 'District', 'Earnings')
rbd_wide = wide_ons_data(ratio_by_district, 'Year', 'District', 'Ratio')
#Combined
region_metrics_wide = pd.merge(pd.merge(pbr_wide, ebr_wide, left_index=True, right_index=True),
                               rbr_wide, left_index=True, right_index=True)
district_metrics_wide = pd.merge(pd.merge(pbd_wide, ebd_wide, left_index=True, right_index=True),
                               rbd_wide, left_index=True, right_index=True)
district_metrics_wide

## Tall data

In [5]:
def flat_ons_data(data_in,time_step,split_by,value_field):
    data_out = data_in.transpose()
    data_out.columns = data_out.iloc[0]
    data_out = data_out[1:].astype(float)
    data_out.index = data_out.index.map(lambda x: str(x)[-4:])
    data_out.index.names = [time_step]
    data_out.columns.names = [split_by]
    data_out = pd.melt(data_out.reset_index(), id_vars=time_step, value_vars=data_out.columns.values.tolist(), var_name=split_by, value_name='Value')
    data_out['Metric'] = value_field
    return data_out

#Region
pbr_flat = flat_ons_data(price_by_region, 'Year', 'Region', 'Price')
ebr_flat = flat_ons_data(earnings_by_region, 'Year', 'Region', 'Earnings')
rbr_flat = flat_ons_data(ratio_by_region, 'Year', 'Region', 'Ratio')
#County
pbd_flat = flat_ons_data(price_by_district, 'Year', 'District', 'Price')
ebd_flat = flat_ons_data(earnings_by_district, 'Year', 'District', 'Earnings')
rbd_flat = flat_ons_data(ratio_by_district, 'Year', 'District', 'Ratio')
#Combined
region_metrics_flat = pd.concat([pbr_flat, ebr_flat, rbr_flat])
district_metrics_flat = pd.concat([pbd_flat, ebd_flat, rbd_flat])

# Plotting

In [ ]:
#Default Colour Palette
palette= sns.color_palette("husl",15)
sns.palplot(palette)

In [ ]:
#Data Calculations
region_metrics_wide['ew_price_pct_dif'] = region_metrics_wide['England and Wales Price'] / region_metrics_wide['England and Wales Price'].iat[0] -1
region_metrics_wide['ew_earnings_pct_dif'] = region_metrics_wide['England and Wales Earnings'] / region_metrics_wide['England and Wales Earnings'].iat[0] -1
#Additional Imports
import matplotlib.ticker as mtick
#Setup
sns.set_style('ticks')
sns.set_palette('husl',2)
fig, ax = plt.subplots(figsize = (16,9))
#ax
sns.lineplot(data=region_metrics_wide, x='Year', y='ew_price_pct_dif', ax=ax, color='black')
sns.lineplot(data=region_metrics_wide, x ='Year', y='ew_earnings_pct_dif', ax=ax, color='#bda800')
ax.set_title('Percentage change in earnings and house prices from 1997 values')
ax.set_xlabel('Year')
ax.yaxis.set_major_formatter(mtick.PercentFormatter(1.0))
ax.set_ylabel('Percentage change from 1997 values')
ax.legend(labels=['House prices', 'Earnings'])
#plt
plt.tight_layout()
plt.savefig("images/House Price and Earnings Percentage Change.png", transparent=False)
plt.show()

In [ ]:
#Setup
sns.set_style('ticks')
sns.set_palette('husl',3)
fig, ax = plt.subplots(figsize = (16,9))
#ax
sns.lineplot(data=region_metrics_wide, x='Year', y='England and Wales Ratio',ax=ax, color="#bda800")
ax.axhline(y=4.5, color='black', linestyle='--')
ax.text(x=20, y=4.6, s='Sole Income Mortgage Limit', fontsize=10)
ax.axhline(y=9.0, color='black', linestyle='--')
ax.text(x=3, y=8.8, s='Joint Income Mortgage Limit', fontsize=10)
ax.set_title('House Price to Earnings Ratio Since 1997')
ax.set_xlabel('Year')
ax.set_ylabel('House Price to Earnings Ratio')
#plt
plt.tight_layout()
plt.savefig("images/House Price and Earnings Ratio.png", transparent=False)
plt.show()

In [ ]:
#Setup
sns.set_style('ticks')
sns.set_palette('tab10')
fig, ax = plt.subplots(figsize = (16, 9))
#ax
sns.lineplot(data=region_metrics_flat[(region_metrics_flat['Metric']=='Ratio')
                                      & (~region_metrics_flat.Region.isin(['England and Wales', 'England']))], x='Year', y='Value', hue='Region', ax=ax)
ax.set_title('Region Variance in Ratio of Earnings to House Prices in England and Wales')
ax.set_ylabel('House Price to Earnings Ratio')
handles,labels = ax.get_legend_handles_labels()
order = [6, 7, 5, 8, 3, 4, 2, 9, 1, 0]
plt.legend([handles[idx] for idx in order],[labels[idx] for idx in order])
#plt
plt.tight_layout()
plt.savefig("images/Region Variance in House Price and Earnings Ratio.png", transparent=False)
plt.show()

In [ ]:
#data tranformations
region_metrics_flat['Year'] = region_metrics_flat['Year'].astype(int)
district_metrics_flat['Year'] = district_metrics_flat['Year'].astype(int)
#Setup
sns.set_style('ticks')
sns.set_palette('tab10')
fig, (ax1, ax2) = plt.subplots(nrows=1, ncols=2, figsize = (16, 9))
#ax1
sns.histplot(data=region_metrics_flat[(region_metrics_flat['Year'] == 2019)
                                      & (region_metrics_flat['Metric'] == 'Ratio')
                                      & (~region_metrics_flat.Region.isin(['England and Wales', 'England']))]
             , x='Value', ax=ax1, legend=False, color='#bda800')
ax1.axvline(x=8.93, color='black', linestyle='--')
ax1.text(x=9, y=3.5, s='England and Wales Median')
ax1.set_xlabel('House Price to Earnings Ratio')
ax1.set_title('Region Variance in Ratio of Earnings to House Prices in England and Wales')
#ax2
sns.histplot(data=district_metrics_flat[(district_metrics_flat['Year'] == 2019) & (district_metrics_flat['Metric'] == 'Ratio')]
             , x='Value', ax=ax2, legend=False, color='#bda800')
ax2.axvline(x=8.93, color='black', linestyle='--')
ax2.text(x=9.5, y=55, s='England and Wales Median')
ax2.set_xlabel('House Price to Earnings Ratio')
ax2.set_title('District Variance in Ratio of Earnings to House Prices in England and Wales')
#plt
plt.tight_layout()
plt.savefig("images/Histogram of House Price and Earnings Ratio.png", transparent=False)
plt.show()

In [ ]:
district_mapping_data = district_metrics_flat[(district_metrics_flat['Metric'] == 'Ratio') & (district_metrics_flat['Year'] == 2019)]
district_mapping_data.set_index('District')
district_mapping_data.head()

In [ ]:
fig = px.choropleth(data_frame=district_metrics_flat[(district_metrics_flat['Metric'] == 'Ratio') & (district_metrics_flat['Year'] == 2020)],
                    geojson=uk_districts,
                    locations='District',
                    featureidkey='properties.LAD21NM',
                    color='Value',
                    color_continuous_scale=["black","grey","yellow"],
                    range_color=(3,15),
                    labels={'Value':''},
                    title='House Price to Earnings Ratio by District 2020')
fig.update_geos(visible=False,
                fitbounds='locations')
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_image("images/House Price to Earnings Ratio by District 2020.png")
fig.show()

In [ ]:
fig = px.choropleth(data_frame=district_metrics_flat[(district_metrics_flat['Metric'] == 'Ratio') & (district_metrics_flat['Year'] == 2015)],
                    geojson=uk_districts,
                    locations='District',
                    featureidkey='properties.LAD21NM',
                    color='Value',
                    color_continuous_scale=["black","grey","yellow"],
                    range_color=(3,15),
                    labels={'Value':''},
                    title='House Price to Earnings Ratio by District 2015')
fig.update_geos(visible=False,
                fitbounds='locations')
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_image("images/House Price to Earnings Ratio by District 2015.png")
fig.show()

In [ ]:
fig = px.choropleth(data_frame=district_metrics_flat[(district_metrics_flat['Metric'] == 'Ratio') & (district_metrics_flat['Year'] == 2010)],
                    geojson=uk_districts,
                    locations='District',
                    featureidkey='properties.LAD21NM',
                    color='Value',
                    color_continuous_scale=["black","grey","yellow"],
                    range_color=(3,15),
                    labels={'Value':''},
                    title='House Price to Earnings Ratio by District 2010')
fig.update_geos(visible=False,
                fitbounds='locations')
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_image("images/House Price to Earnings Ratio by District 2010.png")
fig.show()

In [ ]:
fig = px.choropleth(data_frame=district_metrics_flat[(district_metrics_flat['Metric'] == 'Ratio') & (district_metrics_flat['Year'] == 2005)],
                    geojson=uk_districts,
                    locations='District',
                    featureidkey='properties.LAD21NM',
                    color='Value',
                    color_continuous_scale=["black","grey","yellow"],
                    range_color=(3,15),
                    labels={'Value':''},
                    title='House Price to Earnings Ratio by District 2005')
fig.update_geos(visible=False,
                fitbounds='locations')
fig.update_layout(margin={"r":0,"t":40,"l":0,"b":0})
fig.write_image("images/House Price to Earnings Ratio by District 2005.png")
fig.show()